In [1]:
!pip install lxml
import pandas as pd
import numpy as np
import lxml
import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 5.5MB 5.6MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing tran

In [2]:
saintp = pd.read_html('https://ru.wikipedia.org/wiki/%D0%90%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%BE-%D1%82%D0%B5%D1%80%D1%80%D0%B8%D1%82%D0%BE%D1%80%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%D0%B0')
saintp[0]

,№,район,"население, чел.(2020[2])","площадь, км²","плотность населения, чел./км²"
0,1,Адмиралтейский,159 795,1382,"11 562,6"
1,2,Василеостровский,207 482,2147,96638
2,3,Выборгский,522 746,11552,45252
3,4,Калининский,529 187,4018,"13 170,4"
4,5,Кировский,336 157,4746,7083
5,6,Колпинский,193 839,10225,18957
6,7,Красногвардейский,356 628,5635,63288
7,8,Красносельский,408 026,9049,45091
8,9,Кронштадтский,44 461,1953,22765
9,10,Курортный,79 067,26819,2948


In [3]:
saintp_neighborhoods = pd.DataFrame(saintp[0])
saintp_neighborhoods.drop(['№'], axis=1, inplace=True)
saintp_neighborhoods.rename(columns={'район': 'District', 'население, чел.(2020[2])': 'Population', 'площадь, км²': 'Square kilometers / km²', 'плотность населения, чел./км²': 'Population density, people / km²'}, inplace=True)
saintp_neighborhoods.head()

,District,Population,Square kilometers / km²,"Population density, people / km²"
0,Адмиралтейский,159 795,1382,"11 562,6"
1,Василеостровский,207 482,2147,96638
2,Выборгский,522 746,11552,45252
3,Калининский,529 187,4018,"13 170,4"
4,Кировский,336 157,4746,7083


In [4]:
data = pd.read_csv('https://docs.google.com/spreadsheets/d/' + '1LSKSKs-Nq8-VFAijPjW5eT2IwenfFSdird03SyoXVG4' + '/export?gid=0&format=csv', index_col=0)
data.head() 

,District,Latitude,Longitude
1,Адмиралтейский,59.918593,30.296631
2,Василеостровский,59.940631,30.242780
3,Выборгский,60.050448,30.328696
4,Калининский,59.997685,30.396824
5,Кировский,59.876391,30.257603


In [5]:
spb_neighborhoods = pd.merge(saintp_neighborhoods, data, on='District')
spb_neighborhoods.drop(['Population', 'Square kilometers / km²', 'Population density, people / km²'], axis=1, inplace=True)
spb_neighborhoods.head()

,District,Latitude,Longitude
0,Адмиралтейский,59.918593,30.296631
1,Василеостровский,59.940631,30.242780
2,Выборгский,60.050448,30.328696
3,Калининский,59.997685,30.396824
4,Кировский,59.876391,30.257603


In [6]:
address = 'Saint-Petersburg, RU'

geolocator = Nominatim(user_agent="Russia, Saint-Petersburg")
location = geolocator.geocode(address)
latitude_spb = location.latitude
longitude_spb = location.longitude
print('The geograpical coordinate of Saint-Petersburg are {}, {}.'.format(latitude_spb, longitude_spb))

The geograpical coordinate of Saint-Petersburg are 59.9606739, 30.1586551.


In [7]:
map_spb = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=10)

# add markers to map
for lat, lng, district in zip(spb_neighborhoods['Latitude'], spb_neighborhoods['Longitude'], spb_neighborhoods['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_spb)  
    
map_spb

In [8]:
CLIENT_ID = 'XCQNO55UOBTCFBM20ERFO2UH1ZS1TBMLL3HDOJ4NRSG1WPOW' # your Foursquare ID
CLIENT_SECRET = 'VS1KZHTJLHNVRWYDIYNUYDKY14W13I5PLKM4NNYSE1LQ4EO2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=2000
LIMIT=100

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
spb_venues = getNearbyVenues(names=spb_neighborhoods['District'],
                                 latitudes=spb_neighborhoods['Latitude'],
                                 longitudes=spb_neighborhoods['Longitude']
                                )

Адмиралтейский
Василеостровский
Выборгский
Калининский
Кировский
Колпинский
Красногвардейский
Красносельский
Кронштадтский
Курортный
Московский
Невский
Петроградский
Петродворцовый
Приморский
Пушкинский
Фрунзенский
Центральный


In [11]:
print(spb_venues.shape)
spb_venues.tail()

(1306, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1301,Центральный,59.930908,30.361817,come mode medical,59.941132,30.368190,Health & Beauty Service
1302,Центральный,59.930908,30.361817,St. Martin,59.938213,30.344483,Cocktail Bar
1303,Центральный,59.930908,30.361817,Vinissimo,59.935336,30.342868,Wine Shop
1304,Центральный,59.930908,30.361817,Subzero,59.927148,30.342841,Sushi Restaurant
1305,Центральный,59.930908,30.361817,Кофе на кухне,59.935999,30.343889,Coffee Shop


In [12]:
# one hot encoding
spb_onehot = pd.get_dummies(spb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
spb_onehot['District'] = spb_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [spb_onehot.columns[-1]] + list(spb_onehot.columns[:-1])
spb_onehot.head()

,ATM,Accessories Store,Airport,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit,District
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Адмиралтейский
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Адмиралтейский
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Адмиралтейский
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Адмиралтейский
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Адмиралтейский


In [13]:
spb_onehot.shape

(1306, 244)

In [14]:
spb_grouped = spb_onehot.groupby('District').mean().reset_index()
spb_grouped

,District,ATM,Accessories Store,Airport,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,...,Veterinarian,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,Адмиралтейский,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.02,0.010000,0.000000,0.000000,0.00
1,Василеостровский,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.05,0.010000,...,0.00,0.01,0.010000,0.00,0.01,0.00,0.030000,0.000000,0.000000,0.00
2,Выборгский,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.030000,0.01,0.000000,...,0.01,0.00,0.000000,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.00
3,Калининский,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.000000,...,0.00,0.00,0.000000,0.01,0.00,0.00,0.020000,0.010000,0.010000,0.00
4,Кировский,0.00,0.00,0.000000,0.000000,0.010204,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.00,0.010204,0.010204,0.010204,0.00
5,Колпинский,0.00,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00
6,Красногвардейский,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.011236,0.00,0.011236,...,0.00,0.00,0.011236,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00
7,Красносельский,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00
8,Кронштадтский,0.00,0.00,0.117647,0.000000,0.000000,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00
9,Курортный,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00


In [15]:
spb_grouped.shape

(18, 244)

In [16]:
num_top_venues = 10

for hood in spb_grouped['District']:
    print("----"+hood+"----")
    temp = spb_grouped[spb_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Адмиралтейский----
                venue  freq
0               Hotel  0.09
1         Coffee Shop  0.08
2              Bakery  0.06
3                Café  0.04
4        Concert Hall  0.04
5          Restaurant  0.03
6          Hookah Bar  0.03
7  Italian Restaurant  0.03
8               Plaza  0.03
9       Historic Site  0.02


----Василеостровский----
                venue  freq
0              Bakery  0.07
1                Park  0.05
2         Art Gallery  0.05
3         Coffee Shop  0.05
4        Dance Studio  0.05
5           Wine Shop  0.03
6           Gastropub  0.03
7       Auto Workshop  0.03
8  Italian Restaurant  0.02
9    Doner Restaurant  0.02


----Выборгский----
                  venue  freq
0                Bakery  0.09
1  Gym / Fitness Center  0.05
2          Dance Studio  0.04
3                   Spa  0.03
4            Hookah Bar  0.03
5        Cosmetics Shop  0.03
6        Massage Studio  0.03
7    Salon / Barbershop  0.03
8                Arcade  0.03
9           P

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['District'] = spb_grouped['District']

for ind in np.arange(spb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(spb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Адмиралтейский,Hotel,Coffee Shop,Bakery,Concert Hall,Café,Italian Restaurant,Restaurant,Plaza,Hookah Bar,Opera House
1,Василеостровский,Bakery,Art Gallery,Dance Studio,Park,Coffee Shop,Wine Shop,Auto Workshop,Gastropub,Martial Arts Dojo,Beer Store
2,Выборгский,Bakery,Gym / Fitness Center,Dance Studio,Massage Studio,Salon / Barbershop,Cosmetics Shop,Hookah Bar,Arcade,Spa,Irish Pub
3,Калининский,Park,Bakery,Dessert Shop,Gastropub,Dance Studio,Auto Workshop,Pub,Athletics & Sports,Pet Store,Sporting Goods Shop
4,Кировский,Clothing Store,Bakery,Auto Workshop,Gym / Fitness Center,Sporting Goods Shop,Doner Restaurant,Pier,Café,Hookah Bar,Supplement Shop


In [19]:
# set number of clusters
kclusters = 5

spb_grouped_clustering = spb_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(spb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 1, 3, 2, 4, 3], dtype=int32)

In [20]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

spb_merged = spb_neighborhoods

spb_merged = spb_merged.join(neighborhoods_venues_sorted.set_index('District'), on='District')

spb_merged = spb_merged.dropna()
spb_merged # check the last columns!

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Адмиралтейский,59.918593,30.296631,0,Hotel,Coffee Shop,Bakery,Concert Hall,Café,Italian Restaurant,Restaurant,Plaza,Hookah Bar,Opera House
1,Василеостровский,59.940631,30.242780,0,Bakery,Art Gallery,Dance Studio,Park,Coffee Shop,Wine Shop,Auto Workshop,Gastropub,Martial Arts Dojo,Beer Store
2,Выборгский,60.050448,30.328696,0,Bakery,Gym / Fitness Center,Dance Studio,Massage Studio,Salon / Barbershop,Cosmetics Shop,Hookah Bar,Arcade,Spa,Irish Pub
3,Калининский,59.997685,30.396824,0,Park,Bakery,Dessert Shop,Gastropub,Dance Studio,Auto Workshop,Pub,Athletics & Sports,Pet Store,Sporting Goods Shop
4,Кировский,59.876391,30.257603,0,Clothing Store,Bakery,Auto Workshop,Gym / Fitness Center,Sporting Goods Shop,Doner Restaurant,Pier,Café,Hookah Bar,Supplement Shop
5,Колпинский,59.775082,30.257603,1,River,Bakery,Airport Service,Paintball Field,Bus Station,Scenic Lookout,Farmers Market,Fabric Shop,Factory,Farm
6,Красногвардейский,59.964458,30.460398,3,Auto Workshop,Convenience Store,Gym / Fitness Center,Gym,Café,Caucasian Restaurant,Pet Store,Supermarket,Snack Place,Sushi Restaurant
7,Красносельский,59.790793,30.121823,2,Convenience Store,Baby Store,Gym / Fitness Center,Shopping Mall,Lake,Grocery Store,Gourmet Shop,Flower Shop,Fast Food Restaurant,Farmers Market
8,Кронштадтский,60.013056,29.714374,4,Historic Site,Surf Spot,Airport,History Museum,Café,Stables,Sports Club,Bus Station,Park,Cosmetics Shop
9,Курортный,60.181218,29.864878,3,Restaurant,Pool,Eastern European Restaurant,Supermarket,Steakhouse,Spa,Hotel,Deli / Bodega,Burger Joint,Lounge


In [21]:
# create map
map_clusters = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(spb_merged['Latitude'], spb_merged['Longitude'], spb_merged['District'], spb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [22]:
# Number of venues per category
venues_per_category = spb_venues[['Venue','Venue Category']].groupby('Venue Category').count().sort_values('Venue', ascending=False)
venues_per_category

,Venue
Venue Category,
Bakery,67
Park,42
Coffee Shop,41
Gym / Fitness Center,34
Auto Workshop,34
...,...
Hunting Supply,1
Hot Dog Joint,1
Hockey Field,1


The most popular cathegories are Bakery, Park and Coffee Shop

In [23]:
# Number of venues per Districts
spb_venues[['District','Venue']].groupby('District').count().sort_values('District')

,Venue
District,
Адмиралтейский,100
Василеостровский,100
Выборгский,100
Калининский,100
Кировский,98
Колпинский,7
Красногвардейский,89
Красносельский,23
Кронштадтский,17


In [24]:
# Number of unique categories
print('The number of unique categories is {}.'.format(len(spb_venues['Venue Category'].unique())))

The number of unique categories is 243.


In [25]:
spb_venues['Venue Category'].unique()

array(['Historic Site', 'Hotel', 'Canal', 'Theater', 'Spa', 'Park',
       'American Restaurant', 'Concert Hall', 'Hotel Bar', 'Opera House',
       'Plaza', 'Indie Movie Theater', 'Hookah Bar', 'Café', 'Dive Shop',
       'Italian Restaurant', 'Music Venue', 'Wine Shop', 'Garden',
       'French Restaurant', 'Bakery', 'Middle Eastern Restaurant',
       'Coffee Shop', 'Mexican Restaurant', 'Hot Dog Joint',
       'Pizza Place', 'Clothing Store', 'Palace', 'Gun Range',
       'Caucasian Restaurant', 'Bed & Breakfast', 'Cycle Studio',
       'Bistro', 'Skating Rink', 'Vegetarian / Vegan Restaurant', 'Bar',
       'Electronics Store', 'Flower Shop', 'Wine Bar', 'Donut Shop',
       'Health Food Store', 'Restaurant', 'Museum', 'Climbing Gym',
       'Japanese Restaurant', 'History Museum', 'Gym / Fitness Center',
       'Gym', 'Photography Studio', 'Hostel', 'Tailor Shop', 'Mini Golf',
       'Modern European Restaurant', 'Burger Joint', 'Volleyball Court',
       'Chinese Restaurant', 'G

In [26]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, venue, venueCat in zip(spb_venues['Venue Latitude'], spb_venues['Venue Longitude'], spb_venues['District'], spb_venues['Venue'], spb_venues['Venue Category']):
        label = '{} ({}) - {}'.format(dist, venue, venueCat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [27]:
# Plot the venue 
#latitude = 59.9386
#longitude = 30.3141
map_spb_venues = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=9)
addToMap(spb_venues, 'orange', map_spb_venues)
map_spb_venues

In [28]:
#Let's analyze the most popular venues
bakery = spb_venues.loc[spb_venues['Venue Category'] == 'Bakery']
park = spb_venues.loc[spb_venues['Venue Category'] == 'Park']
coffee_shop = spb_venues.loc[spb_venues['Venue Category'] == 'Coffee Shop']

In [29]:
# Displaying Bakeries
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, venue, venueCat in zip(bakery['Venue Latitude'], bakery['Venue Longitude'], bakery['District'], bakery['Venue'], bakery['Venue Category']):
        label = '{} ({}) - {}'.format(dist, venue, venueCat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_bakery = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=10)
addToMap(bakery, 'red', map_bakery)
map_bakery

In [30]:
# Top 5 districts of Bakeries
bakery.groupby('District')['Venue'].count().nlargest(5)

District
Выборгский          9
Центральный         8
Василеостровский    7
Московский          7
Адмиралтейский      6
Name: Venue, dtype: int64

In [31]:
# Displaying Parks
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, venue, venueCat in zip(park['Venue Latitude'], park['Venue Longitude'], park['District'], park['Venue'], park['Venue Category']):
        label = '{} ({}) - {}'.format(dist, venue, venueCat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_park = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=10)
addToMap(park, 'yellow', map_park)
map_park

In [32]:
# Top 5 districts of Parks
park.groupby('District')['Venue'].count().nlargest(5)

District
Калининский         7
Московский          6
Василеостровский    5
Невский             5
Пушкинский          4
Name: Venue, dtype: int64

In [33]:
# Displaying Coffee Shops
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, venue, venueCat in zip(coffee_shop['Venue Latitude'], coffee_shop['Venue Longitude'], coffee_shop['District'], coffee_shop['Venue'], coffee_shop['Venue Category']):
        label = '{} ({}) - {}'.format(dist, venue, venueCat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_coffee_shop = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=10)
addToMap(coffee_shop, 'blue', map_coffee_shop)
map_coffee_shop

In [34]:
# Top 5 districts of Coffee Shops
coffee_shop.groupby('District')['Venue'].count().nlargest(5)

District
Центральный         10
Адмиралтейский       8
Василеостровский     5
Петроградский        5
Московский           4
Name: Venue, dtype: int64

In [57]:
# Cluster 1 - Most common venues: Bakeries, Parks and Coffee Shops
cluster_1 = spb_merged.loc[spb_merged['Cluster Labels'] == 0]
cluster_1

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Адмиралтейский,59.918593,30.296631,0,Hotel,Coffee Shop,Bakery,Concert Hall,Café,Italian Restaurant,Restaurant,Plaza,Hookah Bar,Opera House
1,Василеостровский,59.940631,30.242780,0,Bakery,Art Gallery,Dance Studio,Park,Coffee Shop,Wine Shop,Auto Workshop,Gastropub,Martial Arts Dojo,Beer Store
2,Выборгский,60.050448,30.328696,0,Bakery,Gym / Fitness Center,Dance Studio,Massage Studio,Salon / Barbershop,Cosmetics Shop,Hookah Bar,Arcade,Spa,Irish Pub
3,Калининский,59.997685,30.396824,0,Park,Bakery,Dessert Shop,Gastropub,Dance Studio,Auto Workshop,Pub,Athletics & Sports,Pet Store,Sporting Goods Shop
4,Кировский,59.876391,30.257603,0,Clothing Store,Bakery,Auto Workshop,Gym / Fitness Center,Sporting Goods Shop,Doner Restaurant,Pier,Café,Hookah Bar,Supplement Shop
10,Московский,59.852176,30.323073,0,Bakery,Hookah Bar,Park,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Spa,Dance Studio,Plaza,Flower Shop
12,Петроградский,59.967687,30.281660,0,Coffee Shop,Dance Studio,Gastropub,Bakery,Italian Restaurant,Spa,Park,Theme Park Ride / Attraction,Health Food Store,Restaurant
14,Приморский,60.017715,30.185145,0,Bakery,Wine Shop,Pharmacy,Supermarket,Pizza Place,Liquor Store,Pool,Café,Brewery,Salon / Barbershop
16,Фрунзенский,59.869964,30.390788,0,Arcade,Bakery,Park,Clothing Store,Gym / Fitness Center,Caucasian Restaurant,Coffee Shop,Electronics Store,Sporting Goods Shop,Soccer Field
17,Центральный,59.930908,30.361817,0,Coffee Shop,Bakery,Bar,Cocktail Bar,Bookstore,Italian Restaurant,Yoga Studio,Gastropub,Health Food Store,Hookah Bar


In [58]:
# Displaying 1st cluster (Cluster Labels = 0)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_1['Latitude'], cluster_1['Longitude'], cluster_1['District'], cluster_1['Cluster Labels'], cluster_1['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_1 = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=10)
addToMap(cluster_1, 'red', map_cluster_1)
map_cluster_1

In [60]:
# Cluster 2 - Special venues: River and Airport Service
cluster_2 = spb_merged.loc[spb_merged['Cluster Labels'] == 1]
cluster_2

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Колпинский,59.775082,30.257603,1,River,Bakery,Airport Service,Paintball Field,Bus Station,Scenic Lookout,Farmers Market,Fabric Shop,Factory,Farm


In [63]:
# Displaying 2nd cluster (Cluster Labels = 1)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_2['Latitude'], cluster_2['Longitude'], cluster_2['District'], cluster_2['Cluster Labels'], cluster_2['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_2 = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=9)
addToMap(cluster_2, 'purple', map_cluster_2)
map_cluster_2

In [64]:
# Cluster 3 - Special venues: Convenience Store and Baby Store
cluster_3 = spb_merged.loc[spb_merged['Cluster Labels'] == 2]
cluster_3

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Красносельский,59.790793,30.121823,2,Convenience Store,Baby Store,Gym / Fitness Center,Shopping Mall,Lake,Grocery Store,Gourmet Shop,Flower Shop,Fast Food Restaurant,Farmers Market


In [66]:
# Displaying 3rd cluster (Cluster Labels = 2)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_3['Latitude'], cluster_3['Longitude'], cluster_3['District'], cluster_3['Cluster Labels'], cluster_3['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_3 = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=9)
addToMap(cluster_3, 'green', map_cluster_3)
map_cluster_3

In [67]:
# Cluster 4 - Special venues: Auto Workshops, Restaurant, Café, Historic Site
cluster_4 = spb_merged.loc[spb_merged['Cluster Labels'] == 3]
cluster_4

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Красногвардейский,59.964458,30.460398,3,Auto Workshop,Convenience Store,Gym / Fitness Center,Gym,Café,Caucasian Restaurant,Pet Store,Supermarket,Snack Place,Sushi Restaurant
9,Курортный,60.181218,29.864878,3,Restaurant,Pool,Eastern European Restaurant,Supermarket,Steakhouse,Spa,Hotel,Deli / Bodega,Burger Joint,Lounge
11,Невский,59.881932,30.464602,3,Auto Workshop,Restaurant,Gym / Fitness Center,Park,Doner Restaurant,Dance Studio,Bus Stop,Soccer Field,Supermarket,Convenience Store
13,Петродворцовый,59.877845,29.866549,3,Café,Supermarket,Park,Nightclub,Coffee Shop,Grocery Store,Gym / Fitness Center,Lake,Big Box Store,General Entertainment
15,Пушкинский,59.696674,30.421276,3,Park,Auto Workshop,Café,Historic Site,Dance Studio,Doner Restaurant,Soccer Field,Stables,Gym,Coffee Shop


In [69]:
# Displaying 4th cluster (Cluster Labels = 3)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_4['Latitude'], cluster_4['Longitude'], cluster_4['District'], cluster_4['Cluster Labels'], cluster_4['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_4 = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=9)
addToMap(cluster_4, 'orange', map_cluster_4)
map_cluster_4

In [70]:
# Cluster 5 - Special venues: Historic Site, Surf Spot
cluster_5 = spb_merged.loc[spb_merged['Cluster Labels'] == 4]
cluster_5

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Кронштадтский,60.013056,29.714374,4,Historic Site,Surf Spot,Airport,History Museum,Café,Stables,Sports Club,Bus Station,Park,Cosmetics Shop


In [71]:
# Displaying 5th cluster (Cluster Labels = 4)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_5['Latitude'], cluster_5['Longitude'], cluster_5['District'], cluster_5['Cluster Labels'], cluster_5['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_5 = folium.Map(location=[latitude_spb, longitude_spb], zoom_start=9)
addToMap(cluster_5, 'orange', map_cluster_5)
map_cluster_5